In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import wandb
import seaborn as sns

In [ ]:

def wandb2pd(exp_runs):
    df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    summary_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    config_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    name_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

    summary = [] 
    config = [] 
    name = [] 
    for exp in exp_runs: 
        summary.append(exp.summary._json_dict) 
        config.append({k:v for k,v in exp.config.items() if not k.startswith('_')}) 
        name.append(exp.name)       

    summary_df = pd.DataFrame.from_records(summary) 
    config_df = pd.DataFrame.from_records(config) 
    name_df = pd.DataFrame({'name': name}) 
    df = pd.concat([name_df, config_df, summary_df], axis=1)
    return df

In [ ]:
# colorlist_original = ["#377eb8", "#ff7f00", "#4daf4a", "darkred", "#984ea3", "y", "k", "w"]
# colorlist = [colorlist_original[1], colorlist_original[4]]
colorlist = ["#ff7f00","#984ea3", "y", "k", "w"]

In [ ]:
def create_boxplot_background(metrics, threshold):

    optimizer_list = ['momentum_sgd', 'adam']

    path_list = [
        f'entity_name/project_momentum_sgd',
        f'entity_name/project_adam',
    ]

    box_plot_list = []
    
    for i in range(len(path_list)):

        p = path_list[i]
        o = optimizer_list[i]

        api = wandb.Api()
        exp_runs = api.runs(
            path=p,
            filters={'state':'finished'}
            )

        exp_runs
        df = wandb2pd(exp_runs)
        df = df[df[metrics[0]]>threshold]
        df['gap'] = df[metrics[0]]-df[metrics[1]]
        df['optimizer'] = o

        box_plot_list.append(df)

    df_boxplot = pd.concat(box_plot_list)
    print(df_boxplot.columns)

    plt.clf()
    sns.color_palette(colorlist,2)
    sns.set_palette(colorlist)

    ax = sns.boxplot(x='optimizer', y=metrics[0], data=df_boxplot)
    ax.set_xticklabels(['Momentum', 'Adam'])

    plt.ylim(0, 1.)
    plt.title(f"Background Challenge")
    plt.grid(linewidth=1)
    plt.savefig(f'figs/box/filtered/boxplot_{metrics[0]}_background.pdf')
    plt.show()

    ax = sns.boxplot(x='optimizer', y=metrics[1], data=df_boxplot)
    ax.set_xticklabels(['Momentum', 'Adam'])

    plt.ylim(0, 1.)
    plt.title(f"Background Challenge")
    plt.grid(linewidth=1)
    plt.savefig(f'figs/box/filtered/boxplot_{metrics[1]}_background.pdf')
    plt.show()


    ax = sns.boxplot(x='optimizer', y='gap', data=df_boxplot)
    ax.set_xticklabels(['Momentum', 'Adam'])

    plt.ylim(0, 1.)
    plt.title(f"Background Challenge")
    plt.grid(linewidth=1)
    plt.savefig(f'figs/box/filtered/boxplot_gap_background.pdf')
    plt.show()

In [ ]:
create_boxplot_background(['acc_orig', 'acc_mixed_rand'], 1/10 *2)